In [23]:
from AlignmentMeterLVA import *
%reload_ext autoreload
%autoreload 2

In [8]:
org_alignments_df = pickle.load(open('org_alignments_df.p', 'rb'))
leg_votes_df = pickle.load(open('leg_votes_df.p', 'rb'))

In [9]:
org_alignments_df.head()

,oid,bid,hid,analysis_flag,alignment,name,date
3,8,CA_201520160AB1056,366,0,For,California State Association Of Counties,2015-06-30
4,8,CA_201520160AB1159,130,0,For,California State Association Of Counties,2015-04-13
5,8,CA_201520160AB1193,184,0,Against,California State Association Of Counties,2015-04-28
6,8,CA_201520160AB1238,716,0,For,California State Association Of Counties,2016-01-21
7,8,CA_201520160AB1239,174,0,For,California State Association Of Counties,2015-04-27


In [10]:
leg_votes_df.head()

,pid,voteId,result,state,lastTouched,dr_id,bid,VoteDate,hid,first,middle,last
0,65,39065,For,CA,2016-07-12 23:31:00,235727,CA_201520160AB1,2015-06-10,305,Janet,None,Nguyen
1,92,39065,For,CA,2016-07-12 23:31:00,341579,CA_201520160AB1,2015-06-10,305,Jim,None,Beall
2,99,39065,For,CA,2016-07-12 23:31:00,362651,CA_201520160AB1,2015-06-10,305,Robert,None,Hertzberg
3,103,39065,For,CA,2016-07-12 23:31:00,372163,CA_201520160AB1,2015-06-10,305,Ed,None,Hernandez
4,104,39065,For,CA,2016-07-12 23:31:00,375538,CA_201520160AB1,2015-06-10,305,Fran,None,Pavley


In [31]:
org_alignments_df['date'].max()


datetime.date(2016, 8, 31)

In [35]:
for ((oid, bid), oid_alignments_df) in org_alignments_df.groupby(['oid', 'bid']):
    #print(oid_alignments_df)
    min_date = oid_alignments_df['date'].min()
    # Still technically a dataframe to be used in the join
    alignment_row = oid_alignments_df[oid_alignments_df['date'] == min_date]
    alignment = alignment_row['alignment'].iloc[0]
    alignment_votes_df = leg_votes_df.merge(alignment_row, on=['bid'], suffixes=['_leg', '_org'])
    print(alignment_votes_df.head())
    print(type(alignment_votes_df['VoteDate'].iloc[0]))
    print(type(alignment_votes_df['date'].iloc[0]))
    idx = pd.to_datetime(alignment_votes_df['date']) <= pd.to_datetime(alignment_votes_df['VoteDate'])
    print(idx)
    alignment_votes_df = alignment_votes_df[idx]
    print(min_date)
    print(alignment_votes_df)
    break

   pid  voteId   result state         lastTouched   dr_id                 bid  \
0    6   37342      For    CA 2016-07-12 23:26:52   19847  CA_201520160AB1056   
1    7   37342      For    CA 2016-07-12 23:26:52   23573  CA_201520160AB1056   
2   31   37342  Against    CA 2016-07-12 23:26:52  112172  CA_201520160AB1056   
3   35   37342      For    CA 2016-07-12 23:26:52  124734  CA_201520160AB1056   
4   89   37342      For    CA 2016-07-12 23:26:52  332333  CA_201520160AB1056   

    VoteDate  hid_leg   first middle       last  oid  hid_org  analysis_flag  \
0 2015-04-29      190  Autumn   None      Burke    8      366              0   
1 2015-04-29      190   David   None       Chiu    8      366              0   
2 2015-04-29      190   Kevin   None     Mullin    8      366              0   
3 2015-04-29      190    Marc   None  Steinorth    8      366              0   
4 2015-04-29      190   Patty   None      Lopez    8      366              0   

  alignment                     

In [67]:
def count_alignments(results, org_alignment):
    total_votes = 0
    aligned_votes = 0
    last_val = None
    for i, val in enumerate(results.values):
        #print(i+1)
        #Prevents similar votes after date to be counted more than once
        if val != last_val:
            last_val = val
            total_votes += 1
            if val == org_alignment:
                aligned_votes += 1

    return total_votes, aligned_votes

leg_scores = alignment_votes_df.groupby('pid')['result'].apply(count_alignments, alignment)
leg_scores_df = pd.DataFrame(leg_scores)
leg_scores_df.reset_index(level=0, inplace=True)
leg_scores_df['total_votes'] = leg_scores_df['result'].apply(lambda x: x[0])
leg_scores_df['aligned_votes'] = leg_scores_df['result'].apply(lambda x: x[1])
print(type(leg_scores_df['total_votes'].iloc[0]))
leg_scores_df

<class 'numpy.int64'>


,pid,result,total_votes,aligned_votes
0,25,"(1, 1)",1,1
1,26,"(1, 0)",1,0
2,52,"(1, 1)",1,1
3,57,"(1, 1)",1,1
4,58,"(1, 1)",1,1
5,60,"(1, 1)",1,1
6,62,"(1, 1)",1,1
7,63,"(1, 1)",1,1
8,66,"(1, 1)",1,1
9,68,"(1, 1)",1,1


In [16]:
leg_votes_df.groupby(['pid', 'bid']).size()

pid    bid               
1      CA_201520160AB1018    1
       CA_201520160AB1025    1
       CA_201520160AB1031    1
       CA_201520160AB1033    1
       CA_201520160AB1051    1
       CA_201520160AB1060    1
       CA_201520160AB1102    1
       CA_201520160AB1114    3
       CA_201520160AB1117    1
       CA_201520160AB1129    1
       CA_201520160AB1130    1
       CA_201520160AB1147    1
       CA_201520160AB1149    1
       CA_201520160AB1162    1
       CA_201520160AB1163    1
       CA_201520160AB1172    1
       CA_201520160AB1177    1
       CA_201520160AB1188    1
       CA_201520160AB1194    1
       CA_201520160AB12      1
       CA_201520160AB1211    1
       CA_201520160AB1218    1
       CA_201520160AB1223    1
       CA_201520160AB1231    1
       CA_201520160AB1234    1
       CA_201520160AB1235    1
       CA_201520160AB1261    1
       CA_201520160AB1278    1
       CA_201520160AB1282    1
       CA_201520160AB1291    1
                            ..
10481  CA_201

In [52]:
count = 0
for ((oid, bid), oid_alignments_df) in org_alignments_df.groupby(['oid', 'bid']):
    if len(oid_alignments_df['alignment'].unique()) > 1:
        for ((date, alignment), oid_alignments_df) in oid_alignments_df.groupby(['date', 'alignment']):
            #if date == prev_date:
            #    alignment = no idea
            print(alignment)
            print(date)
        print('')
        if count == 3:
            break
        count+=1

Against
2016-06-21
For
2016-06-28

Against
2015-04-28
For
2015-04-28

For
2015-04-22
Against
2015-07-01

Against
2015-07-15
For
2015-09-10

